In [1]:
import numpy as np

import torch
import torch.nn.functional as F
from torch import nn
from torch import optim
from torch.autograd import Variable
from torchvision import datasets, transforms
from torchsummary import summary

from trainers import trainTorch, advTrain
from evalAttacks import evalClean, evalFGSMEnsem, evalAdvDiffModel, evalCombined

#Different Attacks
from evalAttacks import CWAttackRandomEnsem, CWAttackEnsem

/home/iis-cvl/anaconda3/envs/advAttack/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/iis-cvl/anaconda3/envs/advAttack/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/iis-cvl/anaconda3/envs/advAttack/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/iis-cvl/anaconda3/envs/advAt

/home/iis-cvl/anaconda3/envs/advAttack/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/iis-cvl/anaconda3/envs/advAttack/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/iis-cvl/anaconda3/envs/advAttack/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/iis-cvl

In [2]:
nb_epochs = 2
batch_size = 128
learning_rate = 0.001
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=True, download=True,
                    transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=False, transform=transforms.ToTensor()),
    batch_size=batch_size)

In [3]:
# Loading models from modelZoo
from modelZoo import LeNet5, VGGNet, ResNet
LeNet1 = LeNet5()
LeNet2 = LeNet5()
LeNet3 = LeNet5()
VGG = VGGNet()
ResNet1 = ResNet()
ResNet2 = ResNet()
ResNet3 = ResNet()
ResNet4 = ResNet()

ckptPath = 'ckpts/'
LeNet1.load_state_dict(torch.load(ckptPath + 'LeNet.pth'))
LeNet2.load_state_dict(torch.load(ckptPath + 'LeNet2.pth'))
LeNet3.load_state_dict(torch.load(ckptPath + 'LeNet3.pth'))
VGG.load_state_dict(torch.load(ckptPath + 'VGG.pth'))
ResNet1.load_state_dict(torch.load(ckptPath + 'ResNet1.pth'))
ResNet2.load_state_dict(torch.load(ckptPath + 'ResNet2.pth'))
ResNet3.load_state_dict(torch.load(ckptPath + 'ResNet3.pth'))
ResNet4.load_state_dict(torch.load(ckptPath + 'ResNet4.pth'))

if torch.cuda.is_available():
    LeNet1 = LeNet1.cuda()
    LeNet2 = LeNet2.cuda()
    LeNet3 = LeNet3.cuda()
    VGG = VGG.cuda()
    ResNet1 = ResNet1.cuda()
    ResNet2 = ResNet2.cuda()
    ResNet3 = ResNet3.cuda()
    ResNet4 = ResNet4.cuda()

In [4]:
# Evaluate all models on clean original test data
evalClean(model1=LeNet1, test_loader=test_loader, singleModel=1)
evalClean(model1=LeNet2, test_loader=test_loader, singleModel=1)
evalClean(model1=LeNet3, test_loader=test_loader, singleModel=1)
evalClean(model1=VGG, test_loader=test_loader, singleModel=1)
evalClean(model1=ResNet1, test_loader=test_loader, singleModel=1)
evalClean(model1=ResNet2, test_loader=test_loader, singleModel=1)
evalClean(model1=ResNet3, test_loader=test_loader, singleModel=1)
evalClean(model1=ResNet4, test_loader=test_loader, singleModel=1)

Evaluating single model results on clean data
Clean accuracy: 99.12%
Evaluating single model results on clean data
Clean accuracy: 99.09%
Evaluating single model results on clean data
Clean accuracy: 99.02%
Evaluating single model results on clean data
Clean accuracy: 99.48%
Evaluating single model results on clean data
Clean accuracy: 99.13%
Evaluating single model results on clean data
Clean accuracy: 99.24%
Evaluating single model results on clean data
Clean accuracy: 99.21%
Evaluating single model results on clean data
Clean accuracy: 99.12%


In [5]:
# Evaluate individual models adversarial accuracy
netModels = [LeNet1, ResNet1, VGG]
for i, fgsmNet in enumerate(netModels):
    for j, targetNet in enumerate(netModels):
        print('fgsm: {}, target: {}:'.format(i, j))
        evalAdvDiffModel(fgsmNet, targetNet, test_loader)
print("Evaluating model 0 with Ensemble")
evalFGSMEnsem(netModels[0], [netModels[1], netModels[2]], test_loader)
print("Evaluating model 1 with Ensemble")
evalFGSMEnsem(netModels[1], [netModels[0], netModels[2]], test_loader)
print("Evaluating model 2 with Ensemble")
evalFGSMEnsem(netModels[2], [netModels[0], netModels[1]], test_loader)

fgsm: 0, target: 0:
Evaluating the adv data with another model
Adv accuracy: 2.809％
fgsm: 0, target: 1:
Evaluating the adv data with another model
Adv accuracy: 51.503％
fgsm: 0, target: 2:
Evaluating the adv data with another model
Adv accuracy: 48.527％
fgsm: 1, target: 0:
Evaluating the adv data with another model
Adv accuracy: 74.733％
fgsm: 1, target: 1:
Evaluating the adv data with another model
Adv accuracy: 12.688％
fgsm: 1, target: 2:
Evaluating the adv data with another model
Adv accuracy: 70.352％
fgsm: 2, target: 0:
Evaluating the adv data with another model
Adv accuracy: 52.265％
fgsm: 2, target: 1:
Evaluating the adv data with another model
Adv accuracy: 58.683％
fgsm: 2, target: 2:
Evaluating the adv data with another model
Adv accuracy: 15.625％
Evaluating model 0 with Ensemble
Adv accuracy: 39.854％
Evaluating model 1 with Ensemble
Adv accuracy: 66.367％
Evaluating model 2 with Ensemble
Adv accuracy: 49.852％


In [6]:
# Evaluate against models with same architecture but different weight initialization
netModels = [ResNet1, ResNet2, ResNet3]
for i, fgsmNet in enumerate(netModels):
    for j, targetNet in enumerate(netModels):
        print('fgsm: {}, target: {}:'.format(i, j))
        evalAdvDiffModel(fgsmNet, targetNet, test_loader)
print("Evaluating model 0 with Ensemble")
evalFGSMEnsem(netModels[0], [netModels[1], netModels[2]], test_loader)
print("Evaluating model 1 with Ensemble")
evalFGSMEnsem(netModels[1], [netModels[0], netModels[2]], test_loader)
print("Evaluating model 2 with Ensemble")
evalFGSMEnsem(netModels[2], [netModels[0], netModels[1]], test_loader)

fgsm: 0, target: 0:
Evaluating the adv data with another model
Adv accuracy: 12.688％
fgsm: 0, target: 1:
Evaluating the adv data with another model
Adv accuracy: 29.302％
fgsm: 0, target: 2:
Evaluating the adv data with another model
Adv accuracy: 29.777％
fgsm: 1, target: 0:
Evaluating the adv data with another model
Adv accuracy: 31.982％
fgsm: 1, target: 1:
Evaluating the adv data with another model
Adv accuracy: 9.682％
fgsm: 1, target: 2:
Evaluating the adv data with another model
Adv accuracy: 23.981％
fgsm: 2, target: 0:
Evaluating the adv data with another model
Adv accuracy: 35.196％
fgsm: 2, target: 1:
Evaluating the adv data with another model
Adv accuracy: 29.391％
fgsm: 2, target: 2:
Evaluating the adv data with another model
Adv accuracy: 11.007％
Evaluating model 0 with Ensemble
Adv accuracy: 26.434％
Evaluating model 1 with Ensemble
Adv accuracy: 21.737％
Evaluating model 2 with Ensemble
Adv accuracy: 26.958％


In [7]:
advTrain(ResNet1, [ResNet1], train_loader)
advTrain(ResNet2, [ResNet2], train_loader)
advTrain(ResNet3, [ResNet3], train_loader)

Epoch 0, Step [125], Clean Training accuracy: 98.90%
Epoch 0, Step [125], Adv Training accuracy: 66.91%
Epoch 0, Step [250], Clean Training accuracy: 98.81%
Epoch 0, Step [250], Adv Training accuracy: 86.13%
Epoch 0, Step [375], Clean Training accuracy: 99.22%
Epoch 0, Step [375], Adv Training accuracy: 93.20%
Epoch 1, Step [500], Clean Training accuracy: 99.00%
Epoch 1, Step [500], Adv Training accuracy: 93.83%
Epoch 1, Step [625], Clean Training accuracy: 99.30%
Epoch 1, Step [625], Adv Training accuracy: 90.66%
Epoch 1, Step [750], Clean Training accuracy: 99.12%
Epoch 1, Step [750], Adv Training accuracy: 87.51%
Epoch 1, Step [875], Clean Training accuracy: 99.20%
Epoch 1, Step [875], Adv Training accuracy: 92.34%
Epoch 2, Step [1000], Clean Training accuracy: 99.08%
Epoch 2, Step [1000], Adv Training accuracy: 95.27%
Epoch 2, Step [1125], Clean Training accuracy: 99.33%
Epoch 2, Step [1125], Adv Training accuracy: 96.31%
Epoch 2, Step [1250], Clean Training accuracy: 99.28%
Epoch 

Epoch 0, Step [375], Clean Training accuracy: 99.31%
Epoch 0, Step [375], Adv Training accuracy: 94.87%
Epoch 1, Step [500], Clean Training accuracy: 99.02%
Epoch 1, Step [500], Adv Training accuracy: 95.99%
Epoch 1, Step [625], Clean Training accuracy: 99.39%
Epoch 1, Step [625], Adv Training accuracy: 91.12%
Epoch 1, Step [750], Clean Training accuracy: 99.49%
Epoch 1, Step [750], Adv Training accuracy: 92.49%
Epoch 1, Step [875], Clean Training accuracy: 99.28%
Epoch 1, Step [875], Adv Training accuracy: 96.84%
Epoch 2, Step [1000], Clean Training accuracy: 99.14%
Epoch 2, Step [1000], Adv Training accuracy: 97.10%
Epoch 2, Step [1125], Clean Training accuracy: 99.43%
Epoch 2, Step [1125], Adv Training accuracy: 96.59%
Epoch 2, Step [1250], Clean Training accuracy: 99.34%
Epoch 2, Step [1250], Adv Training accuracy: 96.54%
Epoch 2, Step [1375], Clean Training accuracy: 99.32%
Epoch 2, Step [1375], Adv Training accuracy: 96.76%
Epoch 3, Step [1500], Clean Training accuracy: 99.29%
Ep

Epoch 1, Step [625], Clean Training accuracy: 99.57%
Epoch 1, Step [625], Adv Training accuracy: 96.11%
Epoch 1, Step [750], Clean Training accuracy: 99.31%
Epoch 1, Step [750], Adv Training accuracy: 93.87%
Epoch 1, Step [875], Clean Training accuracy: 99.25%
Epoch 1, Step [875], Adv Training accuracy: 94.52%
Epoch 2, Step [1000], Clean Training accuracy: 99.14%
Epoch 2, Step [1000], Adv Training accuracy: 96.92%
Epoch 2, Step [1125], Clean Training accuracy: 99.45%
Epoch 2, Step [1125], Adv Training accuracy: 97.70%
Epoch 2, Step [1250], Clean Training accuracy: 99.34%
Epoch 2, Step [1250], Adv Training accuracy: 96.48%
Epoch 2, Step [1375], Clean Training accuracy: 99.29%
Epoch 2, Step [1375], Adv Training accuracy: 98.15%
Epoch 3, Step [1500], Clean Training accuracy: 99.31%
Epoch 3, Step [1500], Adv Training accuracy: 96.79%
Epoch 3, Step [1625], Clean Training accuracy: 99.44%
Epoch 3, Step [1625], Adv Training accuracy: 95.80%
Epoch 3, Step [1750], Clean Training accuracy: 99.27

In [9]:
# Reevaluation after training
evalClean(model1=ResNet1, test_loader=test_loader, singleModel=1)
evalClean(model1=ResNet2, test_loader=test_loader, singleModel=1)
evalClean(model1=ResNet3, test_loader=test_loader, singleModel=1)
netModels = [ResNet1, ResNet2, ResNet3]
for i, fgsmNet in enumerate(netModels):
    for j, targetNet in enumerate(netModels):
        print('fgsm: {}, target: {}:'.format(i, j))
        evalAdvDiffModel(fgsmNet, targetNet, test_loader)
print("Evaluating model 0 with Ensemble")
evalFGSMEnsem(netModels[0], [netModels[1], netModels[2]], test_loader)
print("Evaluating model 1 with Ensemble")
evalFGSMEnsem(netModels[1], [netModels[0], netModels[2]], test_loader)
print("Evaluating model 2 with Ensemble")
evalFGSMEnsem(netModels[2], [netModels[0], netModels[1]], test_loader)

Evaluating single model results on clean data
Clean accuracy: 98.94%
Evaluating single model results on clean data
Clean accuracy: 99.19%
Evaluating single model results on clean data
Clean accuracy: 98.30%
fgsm: 0, target: 0:
Evaluating the adv data with another model
Adv accuracy: 97.399％
fgsm: 0, target: 1:
Evaluating the adv data with another model
Adv accuracy: 89.784％
fgsm: 0, target: 2:
Evaluating the adv data with another model
Adv accuracy: 71.143％
fgsm: 1, target: 0:
Evaluating the adv data with another model
Adv accuracy: 87.579％
fgsm: 1, target: 1:
Evaluating the adv data with another model
Adv accuracy: 91.001％
fgsm: 1, target: 2:
Evaluating the adv data with another model
Adv accuracy: 77.977％
fgsm: 2, target: 0:
Evaluating the adv data with another model
Adv accuracy: 80.123％
fgsm: 2, target: 1:
Evaluating the adv data with another model
Adv accuracy: 84.642％
fgsm: 2, target: 2:
Evaluating the adv data with another model
Adv accuracy: 96.578％
Evaluating model 0 with Ense